In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

### 1 - PyTorch Implementation with MCE

### 2 - PyTorch Implementation with MSE

### 3 - Manual Gradients and Updates

### 4 - Regularizers